## sklearn

In [1]:
import sklearn.preprocessing as preprocessing


In [5]:
le = preprocessing.LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])

list(le.classes_)

['amsterdam', 'paris', 'tokyo']

In [6]:
le.transform(["tokyo", "tokyo", "paris"]) 



array([2, 2, 1])

In [35]:
lb = preprocessing.LabelBinarizer()
lb.fit(['1','2','3','4','5'])

list(lb.classes_)

['1', '2', '3', '4', '5']

In [38]:
lb.transform(["1", "2", "2"]) 

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0]])

In [32]:
oh = preprocessing.OneHotEncoder()
oh.fit([['1','2','3'],['2','3','4'],['3','4','5']])

oh.n_values_

array([4, 5, 6])

In [33]:
oh.transform(['1','2','5']).toarray()

/home/ds/.pyenv/versions/anaconda2-2.4.1/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.]])

In [ ]:
encoder = sklearn.preprocessing.OneHotEncoder()
label_encoder = sklearn.preprocessing.LabelEncoder()
data_label_encoded = label_encoder.fit_transform(data['category_feature'])
data['category_feature'] = data_label_encoded
data_feature_one_hot_encoded = encoder.fit_transform(data[['category_feature']].as_matrix())

## spark

In [48]:
import findspark
import os
findspark.init()
import pyspark
from pyspark import SparkFiles
from pyspark import sql
from pyspark import SparkConf

from pyspark.sql import SQLContext, HiveContext
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import DataFrameWriter
from pyspark.sql import DataFrameReader
from pyspark import StorageLevel

from pyspark.sql import functions as F
from pyspark.sql.functions import struct
from pyspark.sql import GroupedData
from pyspark.ml.feature import *

In [42]:
conf = SparkConf()
conf.set('spark.executor.memory', '8g')
conf.set('spark.driver.memory', '12g')
conf.set('spark.python.worker.memory', '2g')
conf.set("spark.driver.maxResultSize", "5g")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set('spark.executor.extraJavaOptions', '-XX:+PrintGCDetails -XX:+UseCompressedOops')

conf.setAppName('features transformers')
conf.getAll()

[(u'spark.serializer', u'org.apache.spark.serializer.KryoSerializer'),
 (u'spark.master', u'spark://spark1.ea.lab:7077'),
 (u'spark.driver.memory', u'12g'),
 (u'spark.python.worker.memory', u'2g'),
 (u'spark.submit.pyFiles',
  u'/home/ds/.ivy2/jars/com.databricks_spark-csv_2.10-1.3.0.jar,/home/ds/.ivy2/jars/org.apache.commons_commons-csv-1.1.jar,/home/ds/.ivy2/jars/com.univocity_univocity-parsers-1.5.1.jar'),
 (u'spark.executor.extraJavaOptions',
  u'-XX:+PrintGCDetails -XX:+UseCompressedOops'),
 (u'spark.jars',
  u'file:/home/ds/.ivy2/jars/com.databricks_spark-csv_2.10-1.3.0.jar,file:/home/ds/.ivy2/jars/org.apache.commons_commons-csv-1.1.jar,file:/home/ds/.ivy2/jars/com.univocity_univocity-parsers-1.5.1.jar'),
 (u'spark.executor.memory', u'8g'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.app.name', u'features transformers'),
 (u'spark.driver.maxResultSize', u'5g'),
 (u'spark.files',
  u'file:/home/ds/.ivy2/jars/com.databricks_spark-csv_2.10-1.3.0.jar,file:/home/ds/.ivy2/jars/

In [74]:
try:
    sc.stop()
except:
    print 'spark context not exists'

In [69]:
sc = pyspark.SparkContext(conf=conf)
sqc = pyspark.SQLContext(sc)

In [53]:
df = sqc.createDataFrame([(1, 0, 3)], ["a", "b", "c"])
df.show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  0|  3|
+---+---+---+



In [54]:
vecAssembler = VectorAssembler(inputCols=["a", "b", "c"], outputCol="features")
vecAssembler.transform(df).show()

+---+---+---+-------------+
|  a|  b|  c|     features|
+---+---+---+-------------+
|  1|  0|  3|[1.0,0.0,3.0]|
+---+---+---+-------------+



In [55]:
vecAssembler.setParams(outputCol="freqs").transform(df).show()

+---+---+---+-------------+
|  a|  b|  c|        freqs|
+---+---+---+-------------+
|  1|  0|  3|[1.0,0.0,3.0]|
+---+---+---+-------------+



In [56]:
params = {vecAssembler.inputCols: ["b", "a"], vecAssembler.outputCol: "vector"}
vecAssembler.transform(df, params).show()

+---+---+---+---------+
|  a|  b|  c|   vector|
+---+---+---+---------+
|  1|  0|  3|[0.0,1.0]|
+---+---+---+---------+



In [57]:
df = sqc.createDataFrame([
    (0, "a"),
    (1, "b"),
    (2, "c"),
    (3, "a"),
    (4, "a"),
    (5, "d")
], ["id", "category"])

stringIndexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)

encoder = OneHotEncoder(dropLast=False, inputCol="categoryIndex", outputCol="categoryVec")
encoded = encoder.transform(indexed)
# encoded.show()
dd = encoded.select('categoryVec').collect()

In [58]:
df.show()

+---+--------+
| id|category|
+---+--------+
|  0|       a|
|  1|       b|
|  2|       c|
|  3|       a|
|  4|       a|
|  5|       d|
+---+--------+



In [59]:
indexed.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          1.0|
|  2|       c|          2.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       d|          3.0|
+---+--------+-------------+



In [60]:
encoded.show()

+---+--------+-------------+-------------+
| id|category|categoryIndex|  categoryVec|
+---+--------+-------------+-------------+
|  0|       a|          0.0|(4,[0],[1.0])|
|  1|       b|          1.0|(4,[1],[1.0])|
|  2|       c|          2.0|(4,[2],[1.0])|
|  3|       a|          0.0|(4,[0],[1.0])|
|  4|       a|          0.0|(4,[0],[1.0])|
|  5|       d|          3.0|(4,[3],[1.0])|
+---+--------+-------------+-------------+



In [62]:
from pyspark.mllib.linalg import Vectors
df = sqc.createDataFrame([(Vectors.dense([158, 11]),),
                                 (Vectors.dense([2, 22]),), 
                                 (Vectors.dense([3, 33]),),
                                 (Vectors.dense([158, 44]),),
                                 (Vectors.dense([158, 44]),)], ["a"])
df.show()

+------------+
|           a|
+------------+
|[158.0,11.0]|
|  [2.0,22.0]|
|  [3.0,33.0]|
|[158.0,44.0]|
|[158.0,44.0]|
+------------+



In [63]:

indexer = VectorIndexer(maxCategories=100, inputCol="a", outputCol="categoryVector")
model = indexer.fit(df)

indexed = model.transform(df)
indexed.show()

+------------+--------------+
|           a|categoryVector|
+------------+--------------+
|[158.0,11.0]|     [2.0,0.0]|
|  [2.0,22.0]|     [0.0,1.0]|
|  [3.0,33.0]|     [1.0,2.0]|
|[158.0,44.0]|     [2.0,3.0]|
|[158.0,44.0]|     [2.0,3.0]|
+------------+--------------+



In [64]:
model.categoryMaps

{0: {2.0: 0, 3.0: 1, 158.0: 2}, 1: {11.0: 0, 22.0: 1, 33.0: 2, 44.0: 3}}

In [73]:
from pyspark.ml.feature import ElementwiseProduct
from pyspark.mllib.linalg import Vectors

data = [(Vectors.dense([1.0, 2.0, 3.0]),), (Vectors.dense([4.0, 5.0, 6.0]),)]
df = sqc.createDataFrame(data, ["vector"])

transformer = ElementwiseProduct(scalingVec=Vectors.dense([0.0, 1.0, 2.0]),
                                 inputCol="vector", outputCol="transformedVector")
transformer.transform(df).show()

+-------------+-----------------+
|       vector|transformedVector|
+-------------+-----------------+
|[1.0,2.0,3.0]|    [0.0,2.0,6.0]|
|[4.0,5.0,6.0]|   [0.0,5.0,12.0]|
+-------------+-----------------+



In [76]:
df = sqc.createDataFrame([
     (1.0, 1.0, "a","q"),
     (0.0, 2.0, "b","w"),
     (0.0, 3.0, "a","q"),    
 ], ["y", "x", "cat1","cat2"])
df.show()

df2 = sqc.createDataFrame([
     (1.0, 1.0, "a","q"),
     (0.0, 2.0, "b","w"),
     (0.0, 3.0, "c","z"),    
 ], ["y", "x", "cat1","cat2"])
df2.show()

AttributeError: 'NoneType' object has no attribute 'sc'

In [75]:
rf = RFormula(formula="y ~ cat1")
rf.fit(df).transform(df).show()

rf = RFormula(formula="y ~ cat2")
rf.fit(df).transform(df).show()

rf = RFormula(formula="y ~ cat1:x")
rf.fit(df).transform(df).show()

rf = RFormula(formula="y ~ cat2:x")
rf.fit(df).transform(df).show()

rf = RFormula(formula="y ~ cat1:x + cat2:x")
rf.fit(df).transform(df).show()

IllegalArgumentException: u'Field "cat1" does not exist.'